## Libraries

In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
pd.set_option('display.max_columns', 100)
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import LabelEncoder
from itertools import combinations

## Load Data and perform basic processing

In [2]:
def process_data(DATA_DIR):
    
    train = pd.read_csv(DATA_DIR+"train_s3TEQDk.csv")
    test = pd.read_csv(DATA_DIR+"test_mSzZ8RL.csv")
    
    #Removes train rows which has Region_Code not present in test set
    test_region_list=test['Region_Code'].tolist()
    train=train[train['Region_Code'].isin(test_region_list)]
    
    train['train_or_test']='train'
    test['train_or_test']='test'
    df=pd.concat([train,test])
    
     
    le = LabelEncoder()
    for col in ['Gender', 'Region_Code', 'Occupation', 'Channel_Code', 'Credit_Product', 'Is_Active']:
        df[col]=  df[col].astype('str')
        df[col]= le.fit_transform(df[col])
        

    
    return train,test,df

### Feature Engineering

In [3]:
def frequency_encoding(column_name,output_column_name,df):
    fe_pol = (df.groupby(column_name).size()) / len(df)
    df[output_column_name] = df[column_name].apply(lambda x : fe_pol[x])


def feature_engineering(df):
    le = LabelEncoder()
    
     #Interaction Feature (Combining 2 categorical features and performing frequency encoding)
        
    cat_features=[]
    le_features=[]
    columns=['Gender', 'Region_Code', 'Occupation', 'Channel_Code', 'Credit_Product', 'Is_Active']

    comb = combinations(columns, 2) 

    for i in list(comb):  
        df[f'{i[0]}_{i[1]}']=df[i[0]].astype(str)+'_'+df[i[1]].astype(str)
        df[f'{i[0]}_{i[1]}_le']=le.fit_transform(df[f'{i[0]}_{i[1]}'])
        le_features.append(f'{i[0]}_{i[1]}_le')
        frequency_encoding(f'{i[0]}_{i[1]}',f'{i[0]}_{i[1]}',df)
        cat_features.append(f'{i[0]}_{i[1]}')   
        
    #Frequency Encoding
    
    frequency_encoding('Region_Code','Region_Code_fe',df)
    frequency_encoding('Channel_Code','Channel_Code_fe',df)
    frequency_encoding('Credit_Product','Credit_Product_fe',df)
    
    #Deriving characteristics of each region by creating aggregate features
    
    regioncode_aggregate_features = df.groupby(['Region_Code']).agg({'Age': ['mean', 'max', 'min','std'],
                                                     'Vintage': ['mean', 'max', 'min','std'], 
                                                     'Avg_Account_Balance': ['mean', 'max', 'min','std'],
                                                     'Gender': ['nunique','count'], 
                                                     'Occupation': ['nunique','count'],
                                                     'Channel_Code': ['nunique','count'] ,
                                                     'Credit_Product': ['nunique','count'] ,
                                                     'Is_Active': ['nunique','count'] ,
                                                      })
    regioncode_aggregate_features.columns = ['regioncode_aggregate_features' + '_'.join(c).strip('_') for c in regioncode_aggregate_features.columns]
    df = pd.merge(df, regioncode_aggregate_features, on = ['Region_Code'], how='left')

    region_occupation_aggregate_features = df.groupby(['Region_Code','Occupation']).agg({'Age': ['mean', 'max', 'min','std'],
                                                     'Vintage': ['mean', 'max', 'min','std'], 
                                                     'Avg_Account_Balance': ['mean', 'max', 'min','std'],
                                                     'Gender': ['nunique','count'], 
                                                     'Channel_Code': ['nunique','count'] ,
                                                     'Credit_Product': ['nunique','count'] ,
                                                     'Is_Active': ['nunique','count'] ,
                                                      })
    region_occupation_aggregate_features.columns = ['region_occupation_aggregate_features' + '_'.join(c).strip('_') for c in region_occupation_aggregate_features.columns]
    df = pd.merge(df, region_occupation_aggregate_features, on = ['Region_Code','Occupation'], how='left')

    region_creditproduct_aggregate_features = df.groupby(['Region_Code','Credit_Product']).agg({'Age': ['mean', 'max', 'min','std'],
                                                     'Vintage': ['mean', 'max', 'min','std'], 
                                                     'Avg_Account_Balance': ['mean', 'max', 'min','std'],
                                                     'Gender': ['nunique','count'], 
                                                     'Occupation': ['nunique','count'],
                                                     'Channel_Code': ['nunique','count'] ,
                                                     'Is_Active': ['nunique','count'] ,
                                                      })
    region_creditproduct_aggregate_features.columns = ['region_creditproduct_aggregate_features' + '_'.join(c).strip('_') for c in region_creditproduct_aggregate_features.columns]
    df = pd.merge(df, region_creditproduct_aggregate_features, on = ['Region_Code','Credit_Product'], how='left')
    
    region_occupationcreditproduct_aggregate_features = df.groupby(['Region_Code','Occupation_Credit_Product']).agg({'Age': ['mean', 'max', 'min','std'],
                                                     'Vintage': ['mean', 'max', 'min','std'], 
                                                     'Avg_Account_Balance': ['mean', 'max', 'min','std'],
                                                     'Gender': ['nunique','count'], 
                                                     'Occupation': ['nunique','count'],
                                                     'Credit_Product': ['nunique','count'] ,
                                                     'Channel_Code': ['nunique','count'],
                                                     'Is_Active': ['nunique','count'] ,
                                                      })

    region_occupationcreditproduct_aggregate_features.columns = ['region_occupationcreditproduct_aggregate_features' + '_'.join(c).strip('_') for c in region_occupationcreditproduct_aggregate_features.columns]
    df = pd.merge(df, region_occupationcreditproduct_aggregate_features, on = ['Region_Code','Occupation_Credit_Product'], how='left')
    
    for i in cat_features:
        df[f'region_{i}_max']=df.groupby('Region_Code')[i].transform('max')
        df[f'region_{i}_min']=df.groupby('Region_Code')[i].transform('min')
        df[f'region_{i}_mean']=df.groupby('Region_Code')[i].transform('mean')
        df[f'region_{i}_std']=df.groupby('Region_Code')[i].transform('std')

    
        df[f'region_occupation_{i}_max']=df.groupby(['Region_Code','Occupation'])[i].transform('max')
        df[f'region_occupation_{i}_min']=df.groupby(['Region_Code','Occupation'])[i].transform('min')
        df[f'region_occupation_{i}_mean']=df.groupby(['Region_Code','Occupation'])[i].transform('mean')
        df[f'region_occupation_{i}_std']=df.groupby(['Region_Code','Occupation'])[i].transform('std')

    
        df[f'region_creditproduct{i}_max']=df.groupby(['Region_Code','Credit_Product'])[i].transform('max')
        df[f'region_creditproduct{i}_min']=df.groupby(['Region_Code','Credit_Product'])[i].transform('min')
        df[f'region_creditproduct{i}_mean']=df.groupby(['Region_Code','Credit_Product'])[i].transform('mean')
        df[f'region_creditproduct{i}_std']=df.groupby(['Region_Code','Credit_Product'])[i].transform('std')
        
    
    #features on gender
    
    creditproduct_aggregate_features = df.groupby(['Credit_Product']).agg({'Age': ['mean', 'max', 'min','std'],
                                                     'Vintage': ['mean', 'max', 'min','std'], 
                                                     'Avg_Account_Balance': ['mean', 'max', 'min','std'],
                                                     'Region_Code': ['nunique','count'], 
                                                     'Occupation': ['nunique','count'],
                                                     'Channel_Code': ['nunique','count'] ,
                                                     'Gender': ['nunique','count'] ,
                                                     'Is_Active': ['nunique','count'] ,
                                                           })
    creditproduct_aggregate_features.columns = ['creditproduct_aggregate_features' + '_'.join(c).strip('_') for c in creditproduct_aggregate_features.columns]
    df = pd.merge(df, creditproduct_aggregate_features, on = ['Credit_Product'], how='left')
        
    occupation_aggregate_features = df.groupby(['Occupation']).agg({'Age': ['mean', 'max', 'min','std'],
                                                     'Vintage': ['mean', 'max', 'min','std'], 
                                                     'Avg_Account_Balance': ['mean', 'max', 'min','std'],
                                                     'Region_Code': ['nunique','count'], 
                                                     'Credit_Product': ['nunique','count'] ,
                                                     'Channel_Code': ['nunique','count'] ,
                                                     'Gender': ['nunique','count'] ,
                                                     'Is_Active': ['nunique','count'] ,
                                                           })
    occupation_aggregate_features.columns = ['occupation_aggregate_features' + '_'.join(c).strip('_') for c in occupation_aggregate_features.columns]
    df = pd.merge(df, occupation_aggregate_features, on = ['Occupation'], how='left')
           

    #features on Channel_Code 
    
    channelcode_aggregate_features = df.groupby(['Channel_Code']).agg({'Age': ['mean', 'max', 'min','std'],
                                                     'Vintage': ['mean', 'max', 'min','std'], 
                                                     'Avg_Account_Balance': ['mean', 'max', 'min','std'],
                                                     'Region_Code': ['nunique','count'], 
                                                     'Occupation': ['nunique','count'],
                                                     'Gender': ['nunique','count'] ,
                                                     'Credit_Product': ['nunique','count'] ,
                                                     'Is_Active': ['nunique','count'] ,
                                                           })
    channelcode_aggregate_features.columns = ['channelcode_aggregate_features' + '_'.join(c).strip('_') for c in channelcode_aggregate_features.columns]
    df = pd.merge(df, channelcode_aggregate_features, on = ['Channel_Code'], how='left')
   
    #Deriving characteristics of Is_Active by creating aggregate features
    
    isactive_aggregate_features = df.groupby(['Is_Active']).agg({'Age': ['mean', 'max', 'min','std'],
                                                     'Vintage': ['mean', 'max', 'min','std'], 
                                                     'Avg_Account_Balance': ['mean', 'max', 'min','std'],
                                                     'Region_Code': ['nunique','count'], 
                                                     'Occupation': ['nunique','count'],
                                                     'Channel_Code': ['nunique','count'] ,
                                                     'Credit_Product': ['nunique','count'] ,
                                                     'Gender': ['nunique','count'] ,
                                                           })
    isactive_aggregate_features.columns = ['isactive_aggregate_features' + '_'.join(c).strip('_') for c in isactive_aggregate_features.columns]
    df = pd.merge(df, isactive_aggregate_features, on = ['Is_Active'], how='left')
   
    print(le_features)
    return df,le_features
   

### Remove unnecessary columns and prepare the train and test data for training

In [4]:
def preparedatafortraining(df,train,test):
    
    train=df.loc[df.train_or_test.isin(['train'])]
    test=df.loc[df.train_or_test.isin(['test'])]
    
    drop_columns={'ID','Is_Lead','train_or_test'}
    
    target=['Is_Lead']
    
    x=train.drop(columns=drop_columns,axis=1)
    y=train[target]
    x_test=test.drop(columns=drop_columns,axis=1)
    
    print(x.shape)
    
    return x,y,x_test

## Save Data

In [5]:
def savedata(**DATA_DIR):
    
    train,test,df=process_data("C:/Users/IDH/Desktop/AV-Job-a-thon/")
    df,cat_features=feature_engineering(df)
    x_train,y_train,x_test=preparedatafortraining(df,train,test)
    
    #x_train.to_pickle("x_train_lgbm.pkl")
    #y_train.to_pickle("y_train_lgbm.pkl")
    #x_test.to_pickle("x_test_lgbm.pkl")
    
    return x_train,y_train,x_test,cat_features

### Train LGBM Model and save the validation and test set prediction for ensembling

In [12]:
def lgbm_model():
    
    x,y,x_test,cat_features=savedata()
   
    
    params={'lambda': 2.8849054495567423, 
        'alpha': 0.001054193185317787, 
        'colsample_bytree': 0.5, 
        'subsample': 0.4, 
        'learning_rate': 0.014, 
        'max_depth': 13, 
        'min_child_weight': 5}
        
    err = [] 

    oofs = np.zeros(shape=(len(x)))
    preds = np.zeros(shape=(len(x_test)))

    Folds=8

    fold = StratifiedKFold(n_splits=Folds, shuffle=True, random_state=2020)
    i = 1

    for train_index, test_index in fold.split(x, y):
        x_train, x_val = x.iloc[train_index], x.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    
        m = LGBMClassifier(n_estimators=10000,verbose= -1,random_state = 24,**params)
    
        m.fit(x_train, y_train,eval_set=[(x_val, y_val)], early_stopping_rounds=30,verbose=False,eval_metric='auc')
    
        pred_y = m.predict_proba(x_val)[:,1]
        oofs[test_index] = pred_y
        print(i, " err_lgm: ", roc_auc_score(y_val,pred_y))
        err.append(roc_auc_score(y_val,pred_y))
        preds+= m.predict_proba(x_test)[:,1]
        i = i + 1
    preds=preds/Folds
    
    print(f"Average StratifiedKFold Score : {sum(err)/Folds} ")
    oof_score = roc_auc_score(y, oofs)
    print(f'\nOOF Auc is : {oof_score}')
    
    oofs=pd.DataFrame(oofs,columns=['lgbmoof'])
    preds=pd.DataFrame(preds,columns=['lgbmpred'])
    
    
    oofs.to_csv('lgbmoof_new.csv',index=False)
    preds.to_csv('lgbmpred_new.csv',index=False)
    

In [13]:
lgbm_model()

['Gender_Region_Code_le', 'Gender_Occupation_le', 'Gender_Channel_Code_le', 'Gender_Credit_Product_le', 'Gender_Is_Active_le', 'Region_Code_Occupation_le', 'Region_Code_Channel_Code_le', 'Region_Code_Credit_Product_le', 'Region_Code_Is_Active_le', 'Occupation_Channel_Code_le', 'Occupation_Credit_Product_le', 'Occupation_Is_Active_le', 'Channel_Code_Credit_Product_le', 'Channel_Code_Is_Active_le', 'Credit_Product_Is_Active_le']
(245725, 394)
[LightGBM] [Warning] lambda_l2 is set with reg_lambda=0.0, will be overridden by lambda=2.8849054495567423. Current value: lambda_l2=2.8849054495567423
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
1  err_lgm:  0.8739888787469909
[LightGBM] [Warning] lambda_l2 is set with reg_lambda=0.0, will be overridden by lambda=2.8849054495567423. Current value: lambda_l2=2.8849054495567423
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^

### Train XGB Model and save the validation and test set prediction for ensembling

In [8]:
def xgb_model():
    
    x,y,x_test,cat_features=savedata()
    
    err = [] 

    oofs = np.zeros(shape=(len(x)))
    preds = np.zeros(shape=(len(x_test)))

    Folds=8

    fold = StratifiedKFold(n_splits=Folds, shuffle=True, random_state=2020)
    i = 1

    for train_index, test_index in fold.split(x, y):
        x_train, x_val = x.iloc[train_index], x.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    
        m = XGBClassifier(n_estimators=10000)
    
        m.fit(x_train, y_train,eval_set=[(x_val, y_val)], early_stopping_rounds=30,verbose=False,eval_metric='auc')
    
        pred_y = m.predict_proba(x_val)[:,1]
        oofs[test_index] = pred_y
        print(i, " err_xgb: ", roc_auc_score(y_val,pred_y))
        err.append(roc_auc_score(y_val,pred_y))
        preds+= m.predict_proba(x_test)[:,1]
        i = i + 1
    preds=preds/Folds
    
    print(f"Average StratifiedKFold Score : {sum(err)/Folds} ")
    oof_score = roc_auc_score(y, oofs)
    print(f'\nOOF Auc is : {oof_score}')
    
    oofs=pd.DataFrame(oofs,columns=['xgboof'])
    preds=pd.DataFrame(preds,columns=['xgbpred'])
    
    oofs.to_csv('xgbmoof2.csv',index=False)
    preds.to_csv('xgbmpred2.csv',index=False)

In [9]:
xgb_model()

['Gender_Region_Code_le', 'Gender_Occupation_le', 'Gender_Channel_Code_le', 'Gender_Credit_Product_le', 'Gender_Is_Active_le', 'Region_Code_Occupation_le', 'Region_Code_Channel_Code_le', 'Region_Code_Credit_Product_le', 'Region_Code_Is_Active_le', 'Occupation_Channel_Code_le', 'Occupation_Credit_Product_le', 'Occupation_Is_Active_le', 'Channel_Code_Credit_Product_le', 'Channel_Code_Is_Active_le', 'Credit_Product_Is_Active_le']
(245725, 394)
1  err_xgb:  0.873611222898492
2  err_xgb:  0.873764121675126
3  err_xgb:  0.8724288209229425
4  err_xgb:  0.8727546816262199
5  err_xgb:  0.8768126367735689
6  err_xgb:  0.873675686244962
7  err_xgb:  0.873525502031608
8  err_xgb:  0.8733482065127425
Average StratifiedKFold Score : 0.8737401098357078 

OOF Auc is : 0.8736795401300403
